In [8]:
!pip install pymongo
import pymongo

from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')  
db = client['Manufacturing'] 

Defaulting to user installation because normal site-packages is not writeable


In [18]:
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go


### 1) Retrieve the count of orders placed by each customer along with their names.


db.orders.aggregate([
  {
    $group: {
      _id: "$o_customerID",
      customerName: { $first: "$customerName" },
      totalOrders: { $sum: 1 }
    }
  },
  {
    $lookup: {
      from: "customer",
      localField: "_id",
      foreignField: "customerID",
      as: "customerDetails"
    }
  },
  {
    $unwind: "$customerDetails"
  },
  {
    $project: {
      _id: 0,
      customerID: "$_id",
      customerName: "$customerDetails.customerName",
      totalOrders: 1
    }
  }
])

In [10]:
collection= db['orders']
pipeline = [
    {
        "$group": {
            "_id": "$o_customerID",
            "customerName": {"$first": "$customerName"},
            "totalOrders": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "customer",
            "localField": "_id",
            "foreignField": "customerID",
            "as": "customerDetails"
        }
    },
    {
        "$unwind": "$customerDetails"
    },
    {
        "$project": {
            "_id": 0,
            "customerID": "$_id",
            "customerName": "$customerDetails.customerName",
            "totalOrders": 1
        }
    }
]

result = list(db.orders.aggregate(pipeline))
res = list(collection.aggregate(pipeline))

df = pd.DataFrame(result)

# Create a bar plot using Plotly Express
fig = px.bar(df, x='customerName', y='totalOrders', labels={'customerName': 'Customer Name', 'totalOrders': 'Total Orders'})
fig.update_layout(
    title='Total Orders per Customer',
    xaxis=dict(title='Customer Name'),
    yaxis=dict(title='Total Orders'),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white')
)
fig.update_xaxes(tickangle=45)  # Rotate x-axis labels for better readability
fig.show()


### 2) Find the total count of products supplied by each supplier and sort the result in descending order.


db.delivery.aggregate([
  {
    $group: {
      _id: "$d_supplierID",
      supplierName: { $first: "$supplierName" },
      totalProductsSupplied: { $sum: 1 }
    }
  },
  {
    $lookup: {
      from: "supplier",
      localField: "_id",
      foreignField: "supplierID",
      as: "supplierDetails"
    }
  },
  {
    $unwind: "$supplierDetails"
  },
  {
    $project: {
      _id: 0,
      supplierID: "$_id",
      supplierName: "$supplierDetails.supplierName",
      totalProductsSupplied: 1
    }
  },
  {
    $sort: { totalProductsSupplied: -1 }
  }
])


In [19]:
collection= db['delivery']

pipeline = [
    {
        "$group": {
            "_id": "$d_supplierID",
            "supplierName": {"$first": "$supplierName"},
            "totalProductsSupplied": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "supplier",
            "localField": "_id",
            "foreignField": "supplierID",
            "as": "supplierDetails"
        }
    },
    {
        "$unwind": "$supplierDetails"
    },
    {
        "$project": {
            "_id": 0,
            "supplierID": "$_id",
            "supplierName": "$supplierDetails.supplierName",
            "totalProductsSupplied": 1
        }
    },
    {
        "$sort": {"totalProductsSupplied": -1}
    }
]

# Run the aggregation pipeline
result = list(db.delivery.aggregate(pipeline))
res = list(collection.aggregate(pipeline))



df = pd.DataFrame(result)


fig = go.Figure(data=[
    go.Bar(x=df['supplierName'], y=df['totalProductsSupplied'])
])

fig.update_layout(
    title='Total Products Supplied per Supplier',
    xaxis=dict(title='Supplier Name'),
    yaxis=dict(title='Total Products Supplied'),
    plot_bgcolor='rgb(17,17,17)',  # Dark background color
    paper_bgcolor='rgb(17,17,17)',  # Dark paper color
    font=dict(color='white'),  # Set text color to white
)

custom_colors = ['red', 'blue', 'green', 'yellow', 'orange']   
fig.update_traces(marker_color=custom_colors)  

# Show the plot
fig.show()


###  3) Retrieve product details and the count of times each product has been ordered.

db.orders.aggregate([
  {
    $group: {
      _id: "$o_plantID",
      productName: { $first: "$productName" },
      productType: { $first: "$productType" },
      totalOrders: { $sum: 1 }
    }
  },
  {
    $lookup: {
      from: "product",
      localField: "_id",
      foreignField: "productID",
      as: "productDetails"
    }
  },
  {
    $unwind: "$productDetails"
  },
  {
    $project: {
      _id: 0,
      productID: "$_id",
      productName: "$productDetails.productName",
      productType: "$productDetails.productType",
      totalOrders: 1
    }
  }
])


In [6]:

pipeline = [
    {
        "$group": {
            "_id": "$o_plantID",
            "productName": {"$first": "$productName"},
            "productType": {"$first": "$productType"},
            "totalOrders": {"$sum": 1}
        }
    },
    {
        "$lookup": {
            "from": "product",
            "localField": "_id",
            "foreignField": "productID",
            "as": "productDetails"
        }
    },
    {
        "$unwind": "$productDetails"
    },
    {
        "$project": {
            "_id": 0,
            "productID": "$_id",
            "productName": "$productDetails.productName",
            "productType": "$productDetails.productType",
            "totalOrders": 1
        }
    }
]

result = list(db.orders.aggregate(pipeline))
df = pd.DataFrame(result)

fig = px.bar(df, x='productName', y='totalOrders', color='productName',
             title='Total Orders per Product', labels={'productName': 'Product Name', 'totalOrders': 'Total Orders'})

fig.update_layout(template='plotly_dark')
fig.show()
